# Geopandas Tutorial

### I. Installation

Install a geopandas enviroment along with its packages:

```console
C:/User/geopandas> conda create --name geo_env python=3.7 notebook pandas numpy geopandas matplotlib descartes

C:/User/geopandas> conda activate geo_env
```

When you're done with the tutorial. You can go back to the base environment by:


```console
C:/User/geopandas> conda deactivate
```

### II. Data Structures

Before proceeding to the rest of the lecture, you must import needed  packages. Run the cell below.

In [ ]:
import pandas as pd
import geopandas
import numpy
import matplotlib.pyplot as plt

A Geoseries is a vector where each entry is a object (shape) corresponding to an observation. Geopandas has three basic classes of geometric objects:

1. Points/ Multi-points
2. Lines / Multi-lines
3. Polygons / Multipolygons

Below, we read a native file included in the geopandas library to show example of a Geoseries. Run the cell below to return a geoseries.

In [ ]:
world = geopandas.read_file(
    geopandas.datasets.get_path('naturalearth_lowres'))
world['geometry']

**Question 1:**
    
Return `world` GEOSERIES index 0-4 using `head()` method taught from lecture/notes from Pandas. You can use the empty cell below to enter the code.

A Geodataframe is a pandas' dataframe that contains a Geoseries. It is its most important property. Below, is a Geodataframe, which is like an attribute table from a GIS software, with difference is that the shape is also part of the table as vector of objects. Run the cell below.

In [ ]:
world.head()

Above is a geodataframe with column `geometry` as the active geoseries. We can run the cell below to plot the geodataframe.

In [ ]:
world.plot()

A geodataframe may contain multiple geoseries but sets only one active geoseries. We can create centroids and make it the active geoseries for our `world` dataframe. The active geoseries will be plotted.

In [ ]:
world['centroids'] = world.centroid
world = world.set_geometry('centroids')
world.plot()

**Question 2:**

Use the empty cell below to set back the active geoseries to `geometry`.

You can return more attributes and methods such as `Geodataframe.centroid` from the above example. Here are some of the most important:

Attributes
- `area` : shape of area (using the units of projection)
- `bounds` : tuple of max and min coordinates for each shape
- `total_bounds` : tuple of max and min coordinates for the whole vector
- `geom_type` : type of geometry
- `is_valid` : test if coordinates make a shape that is reasonable geometric shape
- `crs`: return the Geographic coordinate system used by the Geodataframe

Methods
- `distance` : returns Series with minimum distance from each entry to the other
- `centroid` : returns GeoSeries of centroids
- `representative_point()` : returns GeoSeries of points that are guaranteed to be within each geometry. It does NOT return centroids.
- `to_crs()` : change coordinate reference system. See projections
- `plot()` : plot GeoSeries.

**Question 3:**

Use the cell below to return the coordinate system of `world` geodataframe.

We can use the attribute `area` to compute for the area of each polygon in our active geoseries. Note that the computed area is based on the CRS used.

In [ ]:
world = world.set_geometry('geometry')
world['area'] = world.area
world.head()

**Question 4:**

Use all resources (internet) and the cell below to return the row which has the largest area of `world` without changing its CRS.

### III. Reading and Writing Files

Most shapefiles are compressed into ZIP file. We can directly read a shapefile from it using the example code below.

In [ ]:
zipfile = "zip://data/MuniCities.zip"
munies = geopandas.read_file(zipfile)
munies.head(3)

If we have prior knowledge of the data, we can ignore other fields using the example below. This method can help clean the data.

In [ ]:
munies = geopandas.read_file(zipfile, ignore_fields=["ID_0",
                                                     "ISO",
                                                     "ID_1",
                                                     "NAME_0",
                                                     "VARNAME_2",
                                                     "TYPE_2",
                                                     "NAME_1",
                                                     "ID_2",
                                                     "NL_NAME_2",])
munies.head(3)

We can further filter the data using Pandas tricks. The example below limits our geodataframe to Metro Manila areas only.

In [ ]:
mmcities = munies[munies['PROVINCE'] == 'Metropolitan Manila']
mmcities.head()

We can save or write `mmcities` to a different shapefile. Run the example below to save mmcities to `data/mmcities/mmcities.shp`. Afterwards, check the directory to see the shapefile. You can now directly load it in GIS software such as QGIS.

In [ ]:
mmcities.to_file('data/mmcities/mmcities.shp')

Below is a number of Covid-19 cases in Metro Manila cities. The <a href='https://covid19ph.com/regions/ncr'>data</a> is provided and was last updated in Jul 16, 2020. We can load it using pandas.

In [ ]:
mmcovid = pd.read_csv('data/mmcovid.csv')
mmcovid.head()

**Question 5:**

Suppose we want to create a GeoJSON of Covid-19 cases in Metro Manila, we can merge the mmcities and mmcovid to make a new geodataframe. Using all resources and <a href='https://geopandas.org/io.html#writing-spatial-data'>geopandas write</a>, 
1. merge mmcities and mmcovid to mmdf on NAME_2 and City
2. Drop NAME_2, ENGTYPE_2, REGION in mmdf
3. Rename PROVINCE to Province
4. Save the resulting geodataframe into a GeoJSON: `data/mmdf.geojson`

Use the cell below to show all procedures/codes.

### IV. Making Maps

Mapping in Geopandas is as easy using the `plot()` method on a GeoSeries or GeoDataFrame. We can load an example data.

In [ ]:
zipfile2 = "zip://data/mmcovidshp.zip"
mmcovid_gdf = geopandas.read_file(zipfile2)
mmcovid_gdf.plot()

Resizing the image.

In [ ]:
mmcovid_gdf.plot(figsize=(6, 8))

We can normalize the number of cases to create a Choropleth map to compare the severity of Covid-19 hit in Metro Manila.

In [ ]:
mmcovid_gdf['normalized'] = (mmcovid_gdf['Cases'] - 
                            mmcovid_gdf['Cases'].min()
                           ) / (mmcovid_gdf['Cases'].max() -
                               mmcovid_gdf['Cases'].min())

mmcovid_gdf.plot(column='normalized', figsize=(6, 8))

We can add a legend to our map.

In [ ]:
mmcovid_gdf.plot(column='normalized', figsize=(6, 8), legend=True)

We can change the color ramp of the map.

In [ ]:
mmcovid_gdf.plot(column='normalized', figsize=(6, 8), 
                 legend=True, cmap='OrRd')

We can map multiple layers. Suppose we want to show the centroids of each city on the map. We can copy the original geodataframe and set a different active geoseries. The layers follow an order. The first layer being plotted will be on lowest part. Next layer will be overlayed above the previous plotted layer.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8))
ax.set_aspect('equal')
mmcovid_centroids_gdf = mmcovid_gdf.copy()

mmcovid_centroids_gdf['centroids'] = mmcovid_centroids_gdf.centroid
mmcovid_centroids_gdf = mmcovid_centroids_gdf.set_geometry('centroids')


mmcovid_gdf.plot(ax=ax, column='normalized', 
                 legend=True, cmap='OrRd')
mmcovid_centroids_gdf.plot(ax=ax, color='black', marker='s')

Below are some of the referral hospitals in Metro Manila. Run the cell below to show them. Use this for the next question/activity.

In [ ]:
geojson1 = "data/mmreferralhospitals.geojson"
filename1 = open(geojson1)
mmhospitals = geopandas.read_file(filename1)
mmhospitals.head()

**Question 6**

Using all resources, mmcovid_gdf, mmhospitals and the empty cell below,

1. Compute the normalized 'Recovered' Cases in mmcovid_gdf
2. Make a double layered map
- Use figsize=(6, 8) to resize image
- Use the new computed column to plot the new mmcovid_gdf
- Set cmap='Greens'
- Include a legend
- Plot mmhospitals overlayed above mmcovid_gdf
- Use marker='+' for mmhospitals

Show all procedures/codes.

### V. Geometric Manipulations

Geopandas makes use of all the tools for geometric manipulations using the <a href='https://shapely.readthedocs.io/en/latest/manual.html'>shapely library</a>. Let us load Metro Manila road vector.

In [ ]:
zipfile3 = "zip://data/roads_MM.zip"
mmroads = geopandas.read_file(zipfile3)
mmroads.plot(figsize=(6, 8))

Constructive methods are applied to objects e.g. polygon to manipulate the representation of the object. We can use buffer to lines to enlarge the coverage area of our line objects.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 8))
plt.subplots_adjust(right=1.5)
ax[0].set_title("original")
ax[1].set_title("buffered")
mmroads[0:10].plot(ax=ax[0])
mmroads[0:10].buffer(0.001).plot(ax=ax[1])

Meanwhile, Affine transformation geometrically manipulate the orientation of our objects. Below, we can rotate our lines using an angle as argument. In the example below we rotate the lines counter clockwise by 45 degrees. We can use -45 to make it clockwise.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(6, 8))
plt.subplots_adjust(right=1.5)
ax[0].set_title("original")
ax[1].set_title("45 deg")
ax[2].set_title("-45 deg")
mmroads[0:10].plot(ax=ax[0])
mmroads[0:10].rotate(45).plot(ax=ax[1])
mmroads[0:10].rotate(-45).plot(ax=ax[2])

**Question 7**

Use <a href='https://geopandas.org/geometric_manipulations.html#constructive-methods'>geometric manipulation</a> to only show the boundary of mmcovid_gdf. Use the empty cell below. Use figsize=(6, 8).

### VI. Set Operations with Overlay

When working with multiple spatial datasets – especially multiple polygon or line datasets – users often wish to create new shapes based on places where those datasets overlap (or don’t overlap). These manipulations are often referred using the language of sets – intersections, unions, and differences. 

![image info](./data/overlay_operations.png)

<a href='https://geopandas.org/set_operations.html#set-operations-with-overlay'>source</a>

We can set mmcovid_gdf.boundary as the active geometry and mmroads to show examples.

In [ ]:
mmboundary = mmcovid_gdf.copy()
mmboundary['boundary'] = mmboundary.boundary
mmboundary = mmboundary.set_geometry('boundary')

fig, ax = plt.subplots(1, 2, figsize=(6, 8))
plt.subplots_adjust(right=1.5)
ax[0].set_title("mmcovid_gdf.boundary")
ax[1].set_title("mmroads")
mmboundary.plot(ax=ax[0])
mmroads.plot(ax=ax[1], color='green')

We can illustrate the different overlay modes using the above example. The overlay function will determine the set of all individual geometries from overlaying the two input GeoDataFrames. This result covers the area covered by the two input GeoDataFrames, and also preserves all unique regions defined by the combined boundaries of the two GeoDataFrames.
<a href='https://geopandas.org/set_operations.html#the-different-overlay-operations'>source</a>

By default, overlay only works with two polygons/ multi-polygons. We can accept other vector by stating keep_geom_type argument as False.

Note that succeeding cells will load longer because of the number of data from mmroads.

In [ ]:
from geopandas.tools import overlay

overlay_intersection = overlay(mmcovid_gdf, mmroads,
                                 how='intersection', 
                                 keep_geom_type=False)

overlay_intersection.plot(figsize=(6, 8))

Take note that this is not a clip function. Both mmcovid_gdf and mmroads are preserved to the resulting geodataframe overlay_intersection with respect to both object's intersection.

In [ ]:
overlay_union = overlay(mmcovid_gdf, mmroads,
                                 how='union',
                                 keep_geom_type=False)

overlay_union.plot(figsize=(6, 8))

Using how='difference', order of argument is important. The left argument is the Minuend and the right argument is the Subtrahend. The order thus affect the difference.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 8))
plt.subplots_adjust(right=1.5)
ax[0].set_title("mmcovid_gdf - mmroads")
ax[1].set_title("mmroads - mmcovid_gdf")

overlay_difference_1 = overlay(mmcovid_gdf, mmroads,
                                 how='difference',
                                 keep_geom_type=False)
overlay_difference_2 = overlay(mmroads, mmcovid_gdf,
                                 how='difference',
                                 keep_geom_type=False)

overlay_difference_1.plot(ax=ax[0])
overlay_difference_2.plot(ax=ax[1])

**Question 8**

Use <a href='https://geopandas.org/set_operations.html#the-different-overlay-operations'>overlay operations</a> to show the symmetric difference of mmcovid_gdf and mmroads.

### VII. Aggregation with dissolve

We use pandas' `groupby` function to aggredate data to show summary statistics. But for spatial data, we sometimes need to aggregate geometric features to group them by larger extent.

We load world and group them by continent to show this example.

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
continents = world.dissolve(by='continent')

fig, ax = plt.subplots(2, 1, figsize=(8, 12))
ax[0].set_title("countrie")
ax[1].set_title("continents")
world.plot(ax=ax[0], color='yellow', edgecolor='k')
continents.plot(ax=ax[1], color='lime', edgecolor='k')

We can use `aggfunc='sum'` argument in the `dissolve` function to aggragate numerical data as part of the group by process. Below is an example.

Some other aggfunc inputs: <a href='https://geopandas.org/aggregation_with_dissolve.html#dissolve-arguments'>here</a>.

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
continents = world.dissolve(by='continent', aggfunc='sum')

fig, ax = plt.subplots(2, 1, figsize=(8, 12))
ax[0].set_title("countries")
ax[1].set_title("continents")
world.plot(ax=ax[0], column='pop_est', edgecolor='k')
continents.plot(ax=ax[1], column='pop_est', edgecolor='k')

**Question 9**

Dissolve mmcovid_gdf by `Province`. Show the result below.

### VIII. Summary

**Data Structures**
1. A Geoseries is a vector of shape objects. Classes can vary in the vector.
2. A Geodataframe is a pandas' dataframe with an active Geoseries.
3. Attributes and methods can be applied to a Geoseries and consequently to a Geodataframe.

**Reading and Writting Files**
1. You can read directly through a ZIP file. You can further clean the geodataframe by ignoring fields upon reading.
2. There are many options such as Shape and GeoJSON for writing a geodataframe.
3. A written/saved geodataframe into Shape or any format can be loaded in GIS software.

**Making Maps**
1. Using plot() method, we can display our GeoSeries or GeoDataFrame.
2. There are certain ways to manipulate our map such as resizing, adding legend and choosing different color scheme.
3. We can create multiple layers on the map provided they have a GeoSeries, which means only vectors are allowed (no raster).

**Geometric Manipulation**
1. There are at least two types of geometric manipulation, constructive methods and affine transformation. They are both use to alter the visual representation of the vector objects.

**Set Operations with Overlay**
1. There at least four overlay operations -- union, intersection, symmetric difference and difference.
2. By default we can only overlay polygons. We can set the argument keep_geom_type=False to accept other vector shape.

**Aggregation with dissolve**
1. We use dissolve to aggregate GeoDataFrame geometric features to a greater extent.
2. We can use argfunc argument to aggregate numerical data with our geometric features.

**Congratulations for finishing the Geopandas Tutorial!**

![image info](./data/yoda.jpg)